In [78]:
import os
from pathlib import Path

import numpy as np
import optuna
import pandas as pd
import torch

from mscproject.datasets import CompanyBeneficialOwners
import mscproject.models as mod

while not Path("data") in Path(".").iterdir():
    os.chdir("..")

In [79]:
study = optuna.create_study(
    direction="minimize",
    study_name="pyg_model_selection_final",
    storage="sqlite:///optuna.db",
    load_if_exists=True,
)

# optimise_model_partial = ft.partial(optimise_model, dataset=dataset, device=device)
# study.optimize(optimise_model_partial, n_trials=0)

[I 2022-09-07 09:22:49,903] Using an existing study with name 'pyg_model_selection_final' instead of creating a new one.


In [80]:
# Print top models.
drop_cols = ["number", "datetime_start", "datetime_complete", "duration", "state"]
trials_df: pd.DataFrame = study.trials_dataframe()
trials_df = trials_df.drop(drop_cols, axis=1, errors="ignore")
# trials_df = trials_df.sort_values("user_attrs_aprc", ascending=False)
trials_df = trials_df.sort_values("value")

In [81]:
print("Top models")
top = trials_df.head(10).T
print(top)

Top models
                                  127       126       123       28        128  \
value                        1.069488  1.072288  1.079449  1.082337  1.085265   
params_act                       relu      relu      relu      gelu      relu   
params_bias                     False     False     False     False     False   
params_concat                     NaN       NaN       NaN       NaN       NaN   
params_dropout                0.13138  0.124067  0.194805  0.271818  0.140436   
params_edge_aggr                  max       max       max       max       max   
params_gcn_aggr                   sum       sum       sum       sum       sum   
params_group                      NaN       NaN       NaN       NaN       NaN   
params_han_dropout                NaN       NaN       NaN       NaN       NaN   
params_heads                      NaN       NaN       NaN       NaN       NaN   
params_hidden_channels_log2         8         8         8         7         8   
params_jk        

In [82]:
trials_df.describe().T

,count,mean,std,min,25%,50%,75%,max
value,128.0,1.359136,1.886749,1.069488,1.135250,1.221109,1.243609,22.528276
params_dropout,131.0,0.359638,0.178483,0.025425,0.247511,0.353563,0.441220,0.982209
params_han_dropout,10.0,0.511061,0.360964,0.021622,0.247113,0.511890,0.820367,0.983370
params_heads,28.0,6.464286,5.640345,1.000000,2.000000,4.000000,8.000000,16.000000
params_hidden_channels_log2,131.0,5.877863,1.659578,1.000000,5.000000,6.000000,7.000000,8.000000
params_n_layers,131.0,2.755725,0.691616,1.000000,2.500000,3.000000,3.000000,4.000000
params_negative_slope,10.0,0.558367,0.306372,0.115847,0.352460,0.570279,0.819797,0.959300
params_weight_decay,131.0,0.035605,0.048894,0.000002,0.004856,0.013820,0.040930,0.196133
user_attrs_acc,128.0,0.910085,0.016363,0.726402,0.911531,0.911531,0.911531,0.911531
user_attrs_aprc,128.0,0.147857,0.060011,0.075544,0.095266,0.121935,0.210305,0.268794


In [83]:
# Plot the results.
optuna.visualization.plot_optimization_history(study).show()
optuna.visualization.plot_contour(study).show()
optuna.visualization.plot_slice(study).show()
optuna.visualization.plot_param_importances(study).show()

In [84]:
best = study.best_params

In [85]:
best

{'act': 'relu',
 'bias': False,
 'dropout': 0.13137982265043538,
 'edge_aggr': 'max',
 'gcn_aggr': 'sum',
 'hidden_channels_log2': 8,
 'jk': 'none',
 'model': 'GCN',
 'n_layers': 3,
 'weight_decay': 0.003367973303556277}

In [86]:
dataset_path = "data/pyg/"

# Set the device.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load the dataset.
dataset = CompanyBeneficialOwners(dataset_path, to_undirected=True)
dataset = dataset.data.to(device)

In [87]:
model = mod.get_model("HAN")

In [88]:
model(in_channels=-1, hidden_channels=64, num_layers=3, metadata=dataset.metadata())

KeyError: 'heads'

In [ ]:
dataset

HeteroData(
  company={
    x=[96530, 32],
    y=[96530],
    train_mask=[96530],
    val_mask=[96530],
    test_mask=[96530],
    feature_names=[32]
  },
  person={
    x=[32609, 18],
    y=[32609],
    train_mask=[32609],
    val_mask=[32609],
    test_mask=[32609],
    feature_names=[18]
  },
  (company, owns, company)={
    edge_index=[2, 54607],
    edge_attr=[54607, 1],
    train_mask=[54607],
    val_mask=[54607],
    test_mask=[54607]
  },
  (person, owns, company)={
    edge_index=[2, 80219],
    edge_attr=[80219, 1],
    train_mask=[80219],
    val_mask=[80219],
    test_mask=[80219]
  },
  (company, rev_owns, company)={
    edge_index=[2, 54607],
    edge_attr=[54607, 1],
    train_mask=[54607],
    val_mask=[54607],
    test_mask=[54607]
  },
  (company, rev_owns, person)={
    edge_index=[2, 80219],
    edge_attr=[80219, 1],
    train_mask=[80219],
    val_mask=[80219],
    test_mask=[80219]
  }
)